Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import string
import re
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import gensim.models.keyedvectors as word2vec
import gc
from nltk.tokenize import sent_tokenize
from keras.optimizers import SGD, Adam
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


Using TensorFlow backend.


Data Loading & Processing

In [ ]:
from google.colab import drive
drive.mount('/content/sdrive')

In [ ]:

#train=pd.read_csv('/content/sdrive/My Drive/lplus/train2.tsv',delimiter='\t',encoding='utf-8', header=None)
#test=pd.read_csv('/content/sdrive/My Drive/lplus/test2.tsv',delimiter='\t',encoding='utf-8', header=None)
#valid=pd.read_csv('/content/sdrive/My Drive/lplus/val2.tsv',delimiter='\t',encoding='utf-8', header=None)


train=pd.read_csv('../input/liarplus/liar-plus-master/LIAR-PLUS-master/dataset/train2.tsv',delimiter='\t',encoding='utf-8', header=None)
test=pd.read_csv('../input/liarplus/liar-plus-master/LIAR-PLUS-master/dataset/test2.tsv',delimiter='\t',encoding='utf-8', header=None)
valid=pd.read_csv('../input/liarplus/liar-plus-master/LIAR-PLUS-master/dataset/val2.tsv',delimiter='\t',encoding='utf-8', header=None)

train.columns = ['values','id','label','statement','subject','speaker', 'job', 'state','party','barely_true_c','false_c','half_true_c','mostly_true_c','pants_on_fire_c','venue','extracted_justification']
test.columns = ['values','id','label','statement','subject','speaker', 'job', 'state','party','barely_true_c','false_c','half_true_c','mostly_true_c','pants_on_fire_c','venue','extracted_justification']
valid.columns = ['values','id','label','statement','subject','speaker', 'job', 'state','party','barely_true_c','false_c','half_true_c','mostly_true_c','pants_on_fire_c','venue','extracted_justification']

df = pd.concat([train, valid]) 
len(df)

11524

In [ ]:
print(train.shape)
print(test.shape)
print(valid.shape)

In [ ]:
# function to check distribution of labels
import seaborn as sns
def check_dist(dataset):
  sns.countplot(x='label', data=train, palette='hls')

In [ ]:
asd = pd.DataFrame(train['label'])

In [ ]:
check_dist(train)

In [ ]:

df['text'] = df['statement'] + ' ' + df['extracted_justification'] 
df.head()

,values,id,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue,extracted_justification,text
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...,Says the Annies List political group supports ...
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe...",When did the decline of coal start? It started...
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...,"Hillary Clinton agrees with John McCain ""by vo..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...,Health care reform legislation is likely to ma...
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start...",The economic turnaround started at the end of ...


In [ ]:

test['text'] = test['statement'] + ' ' + test['extracted_justification'] 
test1 = test
test.head() 

,values,id,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue,extracted_justification,text
0,0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,"Meantime, engineering experts agree the wall w...",Building a wall on the U.S.-Mexico border will...
1,1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,She cited layoff notices received by the state...,Wisconsin is on pace to double the number of l...
2,2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,"Trump said that McCain ""has done nothing to he...",Says John McCain has done nothing to help the ...
3,3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,"But spending still goes up. In addition, many ...",Suzanne Bonamici supports a plan that will cut...
4,4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,Our rating A Democratic Party web video making...,When asked by a reporter whether hes at the ce...


In [ ]:

sample = df.text.values[0]
print(sample)

In [ ]:
truth_ = {'false':0,'true':1,'half-true':2 ,'mostly-true':3 ,'barely-true':4 ,'pants-fire':5} # values for labels
df['num_true'] = df['label'].apply(lambda x: truth_[x])
df['num_false'] = df['label'].apply(lambda x: truth_[x])
df['num_half_true'] = df['label'].apply(lambda x: truth_[x])
df['num_mostly_true'] = df['label'].apply(lambda x: truth_[x])
df['num_barely_true'] = df['label'].apply(lambda x: truth_[x])
df['num_pants_fire'] = df['label'].apply(lambda x: truth_[x])
df.head() 

,values,id,label,statement,subject,speaker,job,state,party,barely_true_c,...,pants_on_fire_c,venue,extracted_justification,text,num_true,num_false,num_half_true,num_mostly_true,num_barely_true,num_pants_fire
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,0.0,a mailer,That's a premise that he fails to back up. Ann...,Says the Annies List political group supports ...,0,0,0,0,0,0
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,0.0,a floor speech.,"Surovell said the decline of coal ""started whe...",When did the decline of coal start? It started...,2,2,2,2,2,2
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,9.0,Denver,Obama said he would have voted against the ame...,"Hillary Clinton agrees with John McCain ""by vo...",3,3,3,3,3,3
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,...,44.0,a news release,The release may have a point that Mikulskis co...,Health care reform legislation is likely to ma...,0,0,0,0,0,0
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,...,2.0,an interview on CNN,"Crist said that the economic ""turnaround start...",The economic turnaround started at the end of ...,2,2,2,2,2,2


In [ ]:
test['num_true'] = test['label'].apply(lambda x: truth_[x])
test['num_false'] = test['label'].apply(lambda x: truth_[x])
test['num_half_true'] = test['label'].apply(lambda x: truth_[x])
test['num_mostly_true'] = test['label'].apply(lambda x: truth_[x])
test['num_barely_true'] = test['label'].apply(lambda x: truth_[x])
test['num_pants_fire'] = test['label'].apply(lambda x: truth_[x])
test.head()

,values,id,label,statement,subject,speaker,job,state,party,barely_true_c,...,pants_on_fire_c,venue,extracted_justification,text,num_true,num_false,num_half_true,num_mostly_true,num_barely_true,num_pants_fire
0,0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,...,18,Radio interview,"Meantime, engineering experts agree the wall w...",Building a wall on the U.S.-Mexico border will...,1,1,1,1,1,1
1,1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,...,0,a news conference,She cited layoff notices received by the state...,Wisconsin is on pace to double the number of l...,0,0,0,0,0,0
2,2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,...,61,comments on ABC's This Week.,"Trump said that McCain ""has done nothing to he...",Says John McCain has done nothing to help the ...,0,0,0,0,0,0
3,3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,...,1,a radio show,"But spending still goes up. In addition, many ...",Suzanne Bonamici supports a plan that will cut...,2,2,2,2,2,2
4,4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,...,7,a web video,Our rating A Democratic Party web video making...,When asked by a reporter whether hes at the ce...,5,5,5,5,5,5


In [ ]:

df.text=df.text.astype(str)
test.text=test.text.astype(str)

list_classes = ['num_true', 'num_false', 'num_half_true', 'num_mostly_true','num_barely_true','num_pants_fire'] # six new columns created above
y_t = df[list_classes].values
y_te = test[list_classes].values
max_features = 10

list_sentences_train = df['text']
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)


list_sentences_test = test['text']
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_test))
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

maxlen = 200

X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:

def loadEmbeddingMatrix(typeToLoad):
        if(typeToLoad=="word2vec"):
            word2vecDict = word2vec.KeyedVectors.load_word2vec_format("../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin", binary=True)
            embed_size = 300

        if(typeToLoad=="glove" or typeToLoad=="fasttext" ):
            embeddings_index = dict()
            #Transfer the embedding weights into a dictionary by iterating through every line of the file.
            f = open(EMBEDDING_FILE)
            for line in f:
                #split up line into an indexed array
                values = line.split()
                #first index is word
                word = values[0]
                #store the rest of the values in the array as a new array
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs #50 dimensions
            f.close()
            print('Loaded %s word vectors.' % len(embeddings_index))
        else:
            embeddings_index = dict()
            for word in word2vecDict.wv.vocab:
                embeddings_index[word] = word2vecDict.word_vec(word)
            print('Loaded %s word vectors.' % len(embeddings_index))
            
        gc.collect()
        #We get the mean and standard deviation of the embedding weights so that we could maintain the 
        #same statistics for the rest of our own random generated weights. 
        all_embs = np.stack(list(embeddings_index.values()))
        emb_mean,emb_std = all_embs.mean(), all_embs.std()
        
        nb_words = len(tokenizer.word_index)
        #We are going to set the embedding size to the pretrained dimension as we are replicating it.
        #the size will be Number of Words in Vocab X Embedding Size
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
        gc.collect()

        #With the newly created embedding matrix, we'll fill it up with the words that we have in both 
        #our own dictionary and loaded pretrained embedding. 
        embeddedCount = 0
        for word, i in tokenizer.word_index.items():
            i-=1
            #then we see if this word is in glove's dictionary, if yes, get the corresponding weights
            embedding_vector = embeddings_index.get(word)
            #and store inside the embedding matrix that we will train later on.
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                embeddedCount+=1
        print('total embedded:',embeddedCount,'common words')
        
        del(embeddings_index)
        gc.collect()
        
        #finally, return the embedding matrix
        return embedding_matrix

In [ ]:
embedding_matrix = loadEmbeddingMatrix('word2vec')

Loaded 3000000 word vectors.
total embedded: 8572 common words


In [ ]:

embedding_matrix.shape
from keras.layers import Conv1D
from keras.layers import MaxPooling1D

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from matplotlib import pyplot as plt
from keras.layers import Dense, Embedding, LSTM, GRU,Bidirectional

 Model Architecture
 Using Pretrained Embeddings

In [ ]:
"""inp = Input(shape=(maxlen,)) #maxlen=200 as defined earlier
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)
x = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Bidirectional(LSTM(120, return_sequences=True,name='lstm_layer',dropout=0.2,recurrent_dropout=0.2))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.2)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(6, activation="softmax")(x)
model = Model(inputs=inp, outputs=x)"""

inp = Input(shape=(maxlen,)) #maxlen=200 as defined earlier
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)
#modell.add(embedding_layer)
modell=Dropout(0.2)(x)
modell=Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(modell)
modell=MaxPooling1D(pool_size=2)(modell)
modell=Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(modell)
modell=MaxPooling1D(pool_size=2)(modell)
modell=Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(modell)
modell=BatchNormalization()(modell)
modell=Dense(256, activation='relu')(modell)
modell=Dense(128, activation='relu')(modell)
modell=Dense(64, activation='relu')(modell)
modell=Dense(6, activation='softmax')(modell)
model = Model(inputs=inp, outputs=modell)
#modell.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True) 

model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

In [ ]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 200, 300)          3011700   
_________________________________________________________________
dropout_7 (Dropout)          (None, 200, 300)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 200, 32)           48032     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 100, 32)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 100, 64)           6208      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 50, 64)            0         
__________

In [ ]:
#print(modell.summary())
#modell.fit(X_t, y_t, epochs=50, batch_size=64)

model.save('bilstmmodel.h5')
batch_size = 64
epochs = 50
histr=model.fit(X_t,y_t, batch_size=batch_size, epochs=epochs, validation_data=(X_te,y_te), shuffle=True, verbose=1)

Train on 11524 samples, validate on 1267 samples
Epoch 1/50
11524/11524 [==============================] - 46s 4ms/step - loss: 23.8769 - acc: 0.1779 - val_loss: 23.5017 - val_acc: 0.7214
Epoch 2/50
11524/11524 [==============================] - 42s 4ms/step - loss: 23.8379 - acc: 0.1734 - val_loss: 23.4968 - val_acc: 0.4822
Epoch 3/50
11524/11524 [==============================] - 41s 4ms/step - loss: 23.8350 - acc: 0.1764 - val_loss: 23.4958 - val_acc: 0.4002
Epoch 4/50
11524/11524 [==============================] - 42s 4ms/step - loss: 23.8338 - acc: 0.1906 - val_loss: 23.4954 - val_acc: 0.3915
Epoch 5/50
11524/11524 [==============================] - 42s 4ms/step - loss: 23.8334 - acc: 0.1918 - val_loss: 23.4953 - val_acc: 0.3339
Epoch 6/50
11524/11524 [==============================] - 42s 4ms/step - loss: 23.8330 - acc: 0.1978 - val_loss: 23.4952 - val_acc: 0.2857
Epoch 7/50
11524/11524 [==============================] - 44s 4ms/step - loss: 23.8329 - acc: 0.2003 - val_loss: 23.4

In [ ]:
acc = histr.history['acc']
val_acc = histr.history['val_acc']
loss = histr.history['loss']
val_loss = histr.history['val_loss']

print("Loss: %.3f" % min(loss))
print("Validation Loss: %.3f" % min(val_loss))
print("Accuracy: %.3f" % max(acc))
print("Validation Accuracy: %.3f" % max(val_acc))

Loss: 23.832
Validation Loss: 23.495
Accuracy: 0.203
Validation Accuracy: 0.721
